<a href="https://colab.research.google.com/github/SalvatoreRomano1/net2020/blob/main/Post_Group/Hashtag_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import os


import statsmodels
import statsmodels.api as sm
import scipy.stats as stats

import matplotlib.pyplot as plt

import re
from collections import Counter
from scipy import sparse
import networkx as nx

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# import the csv file with comment and display the pandas dataframe
comm = pd.read_csv(r"/content/gdrive/MyDrive/com_liwc.csv", sep='\t', engine='python')
df = pd.DataFrame(data=comm)

pd.set_option('display.max_columns', None)
df

,Origin_file_order,Site,p_id,dateCreated,p_politician,p_gender,p_GRUPPO_PE,p_LISTA,p_PARTITO,p_governo,p_dx_sx,p_CIRCOSCRIZIONE,p_text,p_favoriteCount,p_shareCount,p_replyCount,p_replyEval,p_numComments,p_numFakeTags,p_rating,p_category,p_topic,p_campagna,p_camapagna2,Target1,Target2,p_targe1-2,target1_s-p,target1_pol,c_text,c_level,c_replyToUser,c_replyToText,c_rating,c_rating3,c_ratingCivile,c_ratingPosNeg,c_category,Unnamed: 38,c_topic,isPersonal,c_WC,c_WPS,c_Sixltr,c_Dic,c_pronomi,c_Io,c_Noi,c_Se,c_Tu,c_Altri,c_Negazio,c_Consen,c_Articol,c_Prepos,c_Numero,c_Affett,c_Sen_Pos,c_Emo_Pos,c_Ottimis,c_Emo_Neg,c_Ansia,c_Rabbia,c_Tristez,c_Mec_Cog,c_Causa,c_Intros,c_Discrep,c_Inibiz,c_possib,c_Certez,c_Proc_Sen,c_Vista,c_Udito,c_Sentim,c_Social,c_Comm,c_Rif_gen,c_amici,c_Famigl,c_Umano,c_Tempo,c_Passato,c_Present,c_Futuro,c_Spazio,c_Sopra,c_Sotto,c_Inclusi,c_Esclusi,c_Movimen,c_Occupaz,c_Scuola,c_Lavoro,c_Raggiun,c_Svago,c_Casa,c_Sport,c_TV_it,c_Musica,c_Soldi,c_Metafis,c_religio,c_Morte,c_Fisico,c_Corpo,c_Sesso,c_Mangiare,c_Dormire,c_Cura_cor,c_parolac,c_Non_flu,c_riempiti,c_Voi,c_Lui_lei,c_Loro,c_Condizio,c_Transiti,c_P_pass,c_gerundio,c_Passivo,c_Essere,c_Avere,c_Formale,c_Io_Ver,c_Tu_Verbo,c_Lui_Verb,c_Noi_Verb,c_Voi_Verb,c_Loro_Ver,c_AllPunc,c_Period,c_Comma,c_Colon,c_SemiC,c_Qmark,c_Exclam,c_Dash,c_Quote,c_Apostro,c_Parenth,c_OtherP,p_WC,p_WPS,p_Sixltr,p_Dic,p_pronomi,p_Io,p_Noi,p_Se,p_Tu,p_Altri,p_Negazio,p_Consen,p_Articol,p_Prepos,p_Numero,p_Affett,p_Sen_Pos,p_Emo_Pos,p_Ottimis,p_Emo_Neg,p_Ansia,p_Rabbia,p_Tristez,p_Mec_Cog,p_Causa,Ip_ntros,p_Discrep,Ip_nibiz,p_possib,p_Certez,p_Proc_Sen,p_Vista,p_Udito,p_Sentim,p_Social,p_Comm,p_Rif_gen,p_amici,p_Famigl,p_Umano,p_Tempo,p_Passato,p_Present,p_Futuro,p_Spazio,p_Sopra,p_Sotto,Ip_nclusi,p_Esclusi,p_Movimen,p_Occupaz,p_Scuola,p_Lavoro,p_Raggiun,p_Svago,p_Casa,p_Sport,p_TV_it,p_Musica,p_Soldi,p_Metafis,p_religio,p_Morte,p_Fisico,p_Corpo,p_Sesso,p_Mangiare,p_Dormire,p_Cura_cor,p_parolac,p_Non_flu,p_riempiti,p_Voi,p_Lui_lei,p_Loro,p_Condizio,p_Transiti,p_P_pass,p_gerundio,p_Passivo,p_Essere,p_Avere,p_Formale,p_Io_Ver,p_Tu_Verbo,p_Lui_Verb,p_Noi_Verb,p_Voi_Verb,p_Loro_Ver,p_AllPunc,p_Period,p_Comma,p_Colon,p_SemiC,p_Qmark,p_Exclam,p_Dash,p_Quote,p_Apostro,p_Parenth,p_OtherP
0,30126,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,destra,s,Se i musulmani pensano di portare la guerra sa...,270,80,57,33,1729,0,problematico,Rifugiati Musulmani,religioni europa,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non politico,C'è poco da dire questa è gente che la guerra ...,1,FRATELLIDITALIA.PUGLIA,Se i musulmani pensano di portare la guerra sa...,problematico,probl-hate,incivile,negativo,Rifugiati Musulmani,NaN,Other,Other,4,2.0,50.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0,0.00,0.00,0,0.00,0.0,0.00,0.00,0.0,0.0,125.00,100.00,0.00,0.0,0.0,0.00,25.00,0.0,0.0,0.00,0.0,0.00,6,6.00,33.33,33.33,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,33.33,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0,0.00,0.00,0,0.00,0.0,0.00,0.0,0.0,0.0,16.67,16.67,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
1,30226,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,destra,s,Se i musulmani pensano di portare la guerra sa...,270,80,57,33,1729,0,problematico,Rifugiati Musulmani,religioni europa,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non p

In [ ]:
#delete the NaN values in dataframe
print('df shape before dropping row: \t', df.shape)
df = df[df['c_text'].notna()]
print('df shape after dropping row: \t', df.shape)
print('Number of Nan in comments text: ', df['c_text'].isna().sum())
df.shape

df shape before dropping row: 	 (78175, 243)
df shape after dropping row: 	 (78174, 243)
Number of Nan in comments text:  0


(78174, 243)

In [ ]:
#dataframe extraction by political side
'''dx_sx = df['p_dx_sx'].unique().tolist()
print(dx_sx)
df_0 = df[df['p_dx_sx'] == 'destra']
df_1 = df[df['p_dx_sx'] == 'sinistra']'''

#dataframe extraction by campaign
campaign = df['p_camapagna2'].unique().tolist()
print(campaign)
df_0 = df[df['p_camapagna2'] == 'Neg-comp']
df_1 = df[df['p_camapagna2'] == 'Positiva']

['Neg-comp', 'Positiva']


In [ ]:
#dataframe whit only comments df_0
comments_0 = df_0[['c_text']].copy()
comments_0.rename(columns={'c_text':'text'}, inplace=True)

#dataframe whit only comments df_1
comments_1 = df_1[['c_text']].copy()
comments_1.rename(columns={'c_text':'text'}, inplace=True)

# Hashtag extraction and dataframe creation

In [ ]:
#return counter of pattern appearance in a text 
def pattern_freq_comment(text, pattern):
    p_freq = Counter()
    p_found= re.findall(pattern, text)
    for p in p_found:
        p_freq[p.lower()] += 1
    return p_freq  

#return counter of pattern appearance in a array of text 
def pattern_freq(docs, pattern):
    p_freq = Counter()
    for text in docs:
        p_found= re.findall(pattern, text)
        for p in p_found:
            p_freq[p.lower()] += 1
    return p_freq 

#select only topic that appears at leat "threshold" times
def hash_extraction(comments,treshold):
  hashtags_patt = re.compile(r'#[\w]*')
  hashtags_freq = pattern_freq(comments['text'].values, hashtags_patt) #hashtag counter with all the hashtags 
  #threshold=2
  hash_dict = {key:val for key, val in hashtags_freq.items() if val > treshold} #we extract the hashtags that appear at minimum threshold times
  return hash_dict


def database_creation(df, hash_dict):
  mask = df.c_text.str.contains('#[\w]*') 
  df_hash = df[mask]
  df_hash = df_hash.reset_index()
  df_hash = df_hash.drop('index',axis = 1)      
  #select subdataframe where at least 1 hastag is contained in the hash_dict (hashtags that appear more than threshold time)
  df_hash_final= df_hash.copy()
  indexes= []
  hashtags_patt = re.compile(r'#[\w]*')
  for index, row in df_hash_final.iterrows():
    present = False
    hash_in_com=pattern_freq_comment(df_hash_final['c_text'][index],hashtags_patt)
    for key in hash_in_com:
        if key in hash_dict.keys():
            present=True
    if not present:
        #print("drop")
        indexes.append(index)

  for i in range(len(indexes)):  
    #df_hash_final = df_hash_final.drop(df_hash_final.index[indexes[i]])
    df_hash_final = df_hash_final.drop([indexes[i]])
  df_hash_final = df_hash_final.reset_index()
  df_hash_final = df_hash_final.drop('index',axis = 1)

  return df_hash_final

## Graph creation

In [ ]:
#biadjacency graph creation
def biadjacency(df,feature):
    emo_dict = dict(zip(feature,[0 for x in range(0,len(feature))]))
    size_df= len(df)
    matrix = np.zeros((len(feature),size_df)) 
    hashtags_patt = re.compile(r'#[\w]*')
    hash_list = list(feature.keys())
    for i in range(size_df):
    
        comm = df['c_text'][i]
        level = df['c_Emo_Neg'][i]
        hashtags_freq_in_com = pattern_freq_comment(comm, hashtags_patt)
        hash = list(hashtags_freq_in_com.keys())
        #print(comm)
        #print(hash)
        for j in range(len(hash)):
            temp = hash[j]
            if hash_list.count(temp) > 0:
              if temp in emo_dict:
                temp_value = emo_dict[temp]
                emo_dict[temp] = temp_value+level
              index = hash_list.index(temp)
              matrix[index][i]+=1 
    matrix_sparse = sparse.csr_matrix(matrix)
    #print(matrix_sparse)
    biadjacency_graph = nx.bipartite.from_biadjacency_matrix(matrix_sparse)
    return biadjacency_graph,emo_dict


#emo_neg normalized dictionary
def emo_neg_dict(emo_dict):
  inverse = [(value, key) for key, value in emotion_dict.items()]
  emo_final = {k: v / total for total in (max(emotion_dict.values()),) for k, v in emotion_dict.items()}
  return emo_final


#bipartite nodes and edges creation
def bipartite_att(df,feature,biad_graph, emo_final): #feature is the hashtag_list
  set1 = pd.DataFrame(feature.keys(),columns =['Label'])
  set1['type']="hashtag"  
  set1['Emo_Neg']= emo_final.values()
  #l = [x for x in range(len(feature1))]
  #set1['Id'] = l

  set2 = df[['c_topic',"p_PARTITO", 'p_politician','c_Emo_Neg','c_Rabbia']]  #change or add attribute attribute 
  set2.reset_index(inplace=True)
  set2['type'] = 'comment'  
  set2.rename(columns={ 'p_PARTITO': 'partito', 'p_politician': 'politician','c_Emo_Neg':'Emo_Neg','c_Rabbia':'Rabbia'}, inplace=True) #change attribute names
  print(len(set1))
  print(len(set2))
  nodes = set1.append(set2)

  l = [x for x in range(len(df)+len(feature))]
  nodes['Id'] = l

  edge = nx.to_pandas_edgelist(biad_graph)

  return nodes,edge


#projection with respect to #
def proj(feature,biad_graph):
  nodes = []
  for i in range(len(feature)):
    nodes.append(i)
  projection = nx.bipartite.projection.weighted_projected_graph(biad_graph, nodes)
  return projection

#projection nodes and edges creation
def proj_att(feature,projection,emo_final): #feature should be the hashtags dictionnary here
  nodes = pd.DataFrame({'Label': feature})
  l = [x for x in range(len(nodes))]
  nodes['Id'] = l
  nodes['Label'] = feature
  
  nodes['Emo_Neg'] = emo_final.values() #added
  edge = nx.to_pandas_edgelist(projection)
  return nodes,edge

In [ ]:
treshold = 2
hash_dict_0 = hash_extraction(comments_0,treshold)
df_0_final = database_creation(df_0, hash_dict_0)

hash_dict_1 = hash_extraction(comments_1,treshold)
df_1_final = database_creation(df_1, hash_dict_1)

In [ ]:
biad_graph, emotion_dict = biadjacency(df_0_final,hash_dict_0)
emo_final = emo_neg_dict(emotion_dict)
nodes, edges = bipartite_att(df_0_final,hash_dict_0,biad_graph,emo_final)

nodes.to_csv("nodelist_bipartite_hash_set0.csv",index=False)
edges.to_csv("edgelist_bipartite_hash_set0.csv",index=False)

160
774


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
biad_graph, emotion_dict = biadjacency(df_0_final,hash_dict_0)
emo_final = emo_neg_dict(emotion_dict)
proj_graph = proj(hash_dict_0,biad_graph)
nodes, edges = proj_att(hash_dict_0,proj_graph,emo_final)

nodes.to_csv("nodelist_projection_hash_set0.csv",index=False)
edges.to_csv("edgelist_projection_hash_set0.csv",index=False)

In [ ]:
biad_graph, emotion_dict = biadjacency(df_1_final,hash_dict_1)
emo_final = emo_neg_dict(emotion_dict)
nodes, edges = bipartite_att(df_1_final,hash_dict_1,biad_graph,emo_final)

nodes.to_csv("nodelist_bipartite_hash_set1.csv",index=False)
edges.to_csv("edgelist_bipartite_hash_set1.csv",index=False)

183
905


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
biad_graph, emotion_dict = biadjacency(df_1_final,hash_dict_1)
emo_final = emo_neg_dict(emotion_dict)
proj_graph = proj(hash_dict_1,biad_graph)
nodes, edges = proj_att(hash_dict_1,proj_graph,emo_final)

nodes.to_csv("nodelist_projection_hash_set1.csv",index=False)
edges.to_csv("edgelist_projection_hash_set1.csv",index=False)